# 🎙️ Week2: Speech-to-Text 강의 노트 (Colab)

> **주의:** Colab 런타임을 재시작하면 `OPENAI_API_KEY`를 다시 설정해야 합니다.

## 진행 순서
1. Quickstart: 환경 준비와 Audio API 개요
2. Transcriptions 엔드포인트 기본 흐름
3. 고급 옵션: 응답 포맷 · 프롬프트 · 로그확률
4. 발화자 구분(Diarization) 워크플로우
5. Translations 엔드포인트로 영어 번역
6. 타임스탬프/스트리밍/긴 오디오 처리 아이디어
7. 품질 향상을 위한 프롬프트/후처리 전략



## 1) Quickstart: 환경 준비와 Audio API 개요
Speech-to-Text(STT)는 Audio API의 `transcriptions`와 `translations` 엔드포인트를 사용해 음성을 텍스트로 변환합니다. 파일 업로드는 25MB 이하로 제한되며 `mp3`, `wav`, `webm` 등을 지원합니다.


In [ ]:
%pip install openai


In [ ]:
import os
from getpass import getpass
from pathlib import Path

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OPENAI_API_KEY: ")
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

print("✅ API 키 설정 완료")


In [ ]:
from openai import OpenAI, AsyncOpenAI

client = OpenAI()
async_client = AsyncOpenAI()
print("✅ 클라이언트 생성 완료")


## 2) Transcriptions 엔드포인트 기본 흐름
- `client.audio.transcriptions.create`로 오디오 파일과 모델을 지정해 텍스트를 얻습니다.
- 추천 모델: `gpt-4o-transcribe`, `gpt-4o-mini-transcribe`
- Whisper 기반 기본 모델: `whisper-1`
- 응답은 기본적으로 JSON(`response.text`)이며, `response_format="text"`로 순수 텍스트를 받을 수 있습니다.



In [ ]:
def transcribe_basic(audio_path):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    with path.open("rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="gpt-4o-transcribe",
            file=audio_file,
        )
    print("전사 결과 예시:\n", transcription.text[:200])

# transcribe_basic("data/sample_lecture.mp3")


## 3) 고급 옵션: 응답 포맷 · 프롬프트 · 로그확률
- `response_format`: `json`, `text`, `verbose_json`, `diarized_json` 등 선택 가능
- `prompt`: 모델이 잘못 인식하는 용어/고유명사를 안내해 정확도 향상
- `include=["logprobs"]`: 토큰별 확률을 확인해 신뢰도를 분석


In [ ]:
def transcribe_with_options(audio_path, *, prompt=None, response_format="json"):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    kwargs = {
        "model": "gpt-4o-transcribe",
        "response_format": response_format,
    }
    if prompt:
        kwargs["prompt"] = prompt
    if response_format == "json":
        kwargs["include"] = ["logprobs"]

    with path.open("rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            file=audio_file,
            **kwargs,
        )

    print("텍스트 예시:", transcription.text[:120])
    if response_format == "json" and getattr(transcription, "logprobs", None):
        print("첫 토큰 로그확률:", transcription.logprobs[0])

# transcribe_with_options("data/sample_lecture.mp3", prompt="This is a talk about GPT-4o and Sora.")


## 4) 발화자 구분(Diarization) 워크플로우
`gpt-4o-transcribe-diarize` 모델은 `response_format="diarized_json"`을 통해 `speaker`, `start`, `end` 정보를 포함한 세그먼트를 제공합니다. 30초 이상 파일은 `chunking_strategy="auto"` 지정을 권장합니다.


In [ ]:
def diarize_demo(audio_path):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    with path.open("rb") as audio_file:
        transcript = client.audio.transcriptions.create(
            model="gpt-4o-transcribe-diarize",
            file=audio_file,
            response_format="diarized_json",
            chunking_strategy="auto",
        )

    for segment in transcript.segments[:5]:
        print(f"{segment.speaker}: {segment.text} ({segment.start:.1f}s ~ {segment.end:.1f}s)")

# diarize_demo("data/meeting.wav")


> **강의 포인트**
> - `known_speaker_names[]`, `known_speaker_references[]`를 이용해 사전에 등록한 음성을 특정 발화자와 매칭할 수 있습니다.
> - 스트리밍(`stream=True`)을 설정하면 세그먼트 완료 시점마다 `transcript.text.segment` 이벤트가 도착합니다.


## 5) Translations 엔드포인트로 영어 번역
`whisper-1` 모델 전용 기능으로, 비영어 음성을 영어 텍스트로 번역합니다. 원문 언어가 무엇이든 출력은 영어입니다.


In [ ]:
def translate_demo(audio_path):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    with path.open("rb") as audio_file:
        translation = client.audio.translations.create(
            model="whisper-1",
            file=audio_file,
        )

    print("영어 번역 결과:", translation.text[:200])

# translate_demo("data/german.mp3")


## 6) 타임스탬프/스트리밍/긴 오디오 처리 아이디어
- `timestamp_granularities=["word"]`: `whisper-1`에서 단어 단위 타임스탬프 추출 (`response_format="verbose_json"`)
- `stream=True`: 완료된 오디오 파일도 스트리밍 응답으로 받아 빠르게 텍스트를 표시
- Realtime API: WebSocket(`wss://api.openai.com/v1/realtime?intent=transcription`)으로 실시간 음성 스트림 처리 가능
- 25MB 초과 파일: PyDub 등으로 분할하고, 문장 경계에서 나누며 이전 세그먼트 텍스트를 prompt로 전달하면 품질 유지에 도움


In [ ]:
def transcribe_with_timestamps(audio_path):
    path = Path(audio_path)
    if not path.exists():
        raise FileNotFoundError(path)

    with path.open("rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file,
            response_format="verbose_json",
            timestamp_granularities=["word"],
        )

    for word_info in transcription.words[:5]:
        print(word_info)

# transcribe_with_timestamps("data/sample_lecture.mp3")


## 7) 품질 향상을 위한 프롬프트/후처리 전략
### Whisper Prompt 활용
- `prompt` 파라미터에 고유명사/약어 리스트를 제공해 인식 정확도를 보정
- 224 토큰 제한으로 목록이 길면 적절히 요약 필요
- 이전 세그먼트 스크립트를 prompt로 전달해 맥락 유지 가능

### GPT-4 기반 후처리
- 전사 결과를 GPT-4에 전달해 오타 수정, 맞춤법 보정, 스타일 통일 가능
- 긴 컨텍스트를 다룰 수 있어 prompt보다 확장성 높음

> **실습 연결**: 학생들에게 prompt 튜닝과 후처리 파이프라인을 직접 실험하게 하고, 품질 향상 전/후 비교를 과제로 제시해 보세요.


> 💡 **강의 마무리**
> - 실습 노트(`04-STT-Practice.ipynb`)와 연계해 기본 전사 → 고급 옵션 → 발화자 구분 → 번역 → 타임스탬프까지 순차적으로 연습하도록 안내하세요.
> - 스트리밍/Realtime API는 데모 시간이 길 수 있으므로, 사전 녹화한 데모 화면이나 로그를 준비해 설명을 보완하면 좋습니다.

